In [1]:
import requests
import numpy as np
import pandas as pd
from tqdm import tqdm
from pprint import pprint

COLLECTION_FILE = 'collection.csv'

collection = pd.read_csv(COLLECTION_FILE)
collection

,card,number,tags
0,4a6f3714-cb21-4056-97db-7449be2c2456,2,hans
1,50ee44a3-09b5-491a-ba51-609756020a31,1,hans
2,3c866bdb-ff94-4f3f-8429-e72c2cbb94ef,1,"hans, dampf"
3,9541810f-1235-4e4a-a334-cb84d623d55e,1,"hans, dampf"


In [4]:
cols_to_copy = ["artist", "cmc", "lang", "name", "set_name"]

for i in tqdm(range(len(collection))):
    row = collection.loc[i]
    res = requests.get(f'https://api.scryfall.com/cards/{row["card"]}')
    data = res.json()

    for col in cols_to_copy:
        collection.loc[i, col] = data[col]

    prices = data["prices"]
    collection.loc[i, ["price_eur", "price_usd", "price_eur_foil", "price_usd_foil"]] = (
        prices["eur"],
        prices["usd"],
        prices["eur_foil"],
        prices["usd_foil"],
    )

collection.head()

100%|██████████| 4/4 [00:15<00:00,  3.92s/it]


,card,number,tags,artist,cmc,lang,name,set_name,price_eur,price_usd,price_eur_foil,price_usd_foil
0,4a6f3714-cb21-4056-97db-7449be2c2456,2,hans,Tyler Jacobson,1.0,en,Opt,The List,0.12,0.14,None,None
1,50ee44a3-09b5-491a-ba51-609756020a31,1,hans,Erion Makuo,1.0,en,Opt,Tales of Middle-earth Commander,0.23,0.16,None,None
2,3c866bdb-ff94-4f3f-8429-e72c2cbb94ef,1,"hans, dampf",Jason Rainville,1.0,en,Shock,Core Set 2020,0.04,0.06,0.37,0.14
3,9541810f-1235-4e4a-a334-cb84d623d55e,1,"hans, dampf",Jon Foster,1.0,es,Shock,Salvat 2011,None,None,None,None


In [5]:
price_matrix = collection.filter(like='price_').astype(float).fillna(0).values
number_matrix = np.diag(collection['number'].values)

total_values = number_matrix @ price_matrix

price_cols = collection.filter(like='price_').columns
collection[price_cols] = total_values

collection.head()

,card,number,tags,artist,cmc,lang,name,set_name,price_eur,price_usd,price_eur_foil,price_usd_foil
0,4a6f3714-cb21-4056-97db-7449be2c2456,2,hans,Tyler Jacobson,1.0,en,Opt,The List,0.24,0.28,0.00,0.00
1,50ee44a3-09b5-491a-ba51-609756020a31,1,hans,Erion Makuo,1.0,en,Opt,Tales of Middle-earth Commander,0.23,0.16,0.00,0.00
2,3c866bdb-ff94-4f3f-8429-e72c2cbb94ef,1,"hans, dampf",Jason Rainville,1.0,en,Shock,Core Set 2020,0.04,0.06,0.37,0.14
3,9541810f-1235-4e4a-a334-cb84d623d55e,1,"hans, dampf",Jon Foster,1.0,es,Shock,Salvat 2011,0.00,0.00,0.00,0.00
